In [22]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from classic_clustering import ClassicClustering
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import preprocessing

# Fazendo os pré-processamentos necessários

In [3]:
#abrindo o csv com os arquivos
df = pd.read_csv('Data.csv',sep='|',encoding='utf8')
textos = list(df['textos'])

In [4]:
#inicializo um objeto da classe que me ajuda com o pré-processamento
cc = ClassicClustering()
#defino as stopwords padrão da classe
cc.define_stop_words()

In [8]:
%%time
#recodifico tudo em utf-8, retiro caracteres especiais e converto tudo para letra minúscula
textos1 = [cc.limpa_utf8(texto.lower()) for texto in textos]

CPU times: user 8.31 s, sys: 128 ms, total: 8.44 s
Wall time: 8.53 s


In [12]:
print(textos1[0][:1000])

portaria/ms no 08, de 10 de abril de 1987 (publicada em dou, de 28 de abril de 1987) o diretor da divisao nacional de vigilancia sanitaria de produtos saneantes domissanitarios, no uso de suas atribuicoes e considerando o risco oferecido pelas formulacoes alaclinas, quando apresentadas sob a forma de liquido premido ou para pulverizacao, que podem provocar serias lesoes em mucosas oculares ou do trato respiratorio, resolve: 1o proibir a fabricacao e comercializacao de saneantes domissanitarios fortemente alcalinos apresentados sob a forma de liquido premido (aerossol), ou liquido para pulverizacao, tais como produtos para limpeza de fornos e desincrustacao de gorduras, de uso domestico. § unico: executam-se do acima disposto os produtos para uso profissional, os quais, nesse caso, devem ser aplicadas por pessoa habilitada, devidamente protegido por roupas, mascaras, oculos e indumetaria apropriados. 2o para efeito da presente sao considerados produtos fortemente alcalinos aqueles conte

In [10]:
%%time
#retiro romanos e stopwords
textos2 = []
for texto in textos1:
    novo_texto = [cc.tira_stopwords_e_romanos(palavra) for palavra in texto.split()]
    textos2.append(' '.join(novo_texto))

CPU times: user 23.3 s, sys: 182 ms, total: 23.5 s
Wall time: 23.7 s


In [11]:
print(textos2[0][:1000])

portaria/ms  08,  10  abril  1987 (publicada  dou,  28  abril  1987)  diretor  divisao nacional  vigilancia sanitaria  produtos saneantes domissanitarios,  uso   atribuicoes  considerando  risco oferecido  formulacoes alaclinas,  apresentadas   forma  liquido premido   pulverizacao,  podem provocar serias lesoes  mucosas oculares   trato respiratorio, resolve: 1o proibir  fabricacao  comercializacao  saneantes domissanitarios fortemente alcalinos apresentados   forma  liquido premido (aerossol),  liquido  pulverizacao, tais  produtos  limpeza  fornos  desincrustacao  gorduras,  uso domestico.  unico: executam-se  acima disposto  produtos  uso profissional,  quais, nesse caso, devem  aplicadas  pessoa habilitada, devidamente protegido  roupas, mascaras, oculos  indumetaria apropriados. 2o  efeito  presente  considerados produtos fortemente alcalinos  contendo bases inorganicas livres  teores acima  1%, inclusive. 3o  produtos alcalinos, assim compreendidos  cuja medida  ph exceda, 8,5, 

In [13]:
%%time
cc.textos_tratados = textos2
cc.stem()
base_tfidf = cc.vec_tfidf()

Comecou a fazer o stemming.
Tempo para fazer o stemming: 143.2124035358429

CPU times: user 2min 25s, sys: 336 ms, total: 2min 25s
Wall time: 2min 26s


In [14]:
print(cc.textos_stem[0][:1000])

portaria/m 08, 10 abril 1987 (public dou, 28 abril 1987) dire divisa nacion vigilanc sanit produt sane domissanitarios, uso atribuico consider risc oferec formulaco alaclinas, apresent form liqu prem pulverizacao, pod provoc ser leso mucos ocul trat respiratorio, resolve: 1o proib fabricaca comercializaca sane domissanitari fort alcalin apresent form liqu prem (aerossol), liqu pulverizacao, tal produt limp forn desincrustaca gorduras, uso domestico. unico: executam-s acim dispost produt uso profissional, quais, ness caso, dev aplic pesso habilitada, devid proteg roupas, mascaras, ocul indumet apropriados. 2o efeit pres consider produt fort alcalin cont bas inorgan livr te acim 1%, inclusive. 3o produt alcalinos, assim compreend cuj med ph exceda, 8,5, fic abrang dispost artig 1o. 4o fic conced praz 90 (noventa) dias, fabric produt registr event dat anteri publicaca pres infrij disposi aqui, contidos, promov recolh unidad exist comercio, find qual mesm recolh fiscalizaca inutilizadas. a

In [36]:
%%time
n_dims = 600
X = cc.SVD(base_tfidf, n_dims)

Começou a redução de dimensionalidade.
Número de dimensões de entrada: 51330
600 dimensões explicam 0.8112234061058099 da variância.
Tempo para fazer a redução de dimensionalidade: 40.887609004974365

CPU times: user 1min 7s, sys: 18.9 s, total: 1min 26s
Wall time: 40.9 s


# Agora vamos rodar os modelos

In [16]:
#gerando labels para as normas (sei que não é a forma mais esperta de se fazer isso)
macrotema_por_norma = list(df['macrotemas'])
macrotemas = list(dict.fromkeys(macrotema_por_norma))

di = dict.fromkeys(macrotema_por_norma)
i=0
for macrotema in macrotemas:
    di[macrotema] = i
    i+=1

y = np.zeros(len(macrotema_por_norma))
i=0
for m in macrotema_por_norma:
    y[i] = di[m]
    i+=1

In [23]:
#normalizando as features com média zero e std 1
#X = preprocessing.scale(X)

In [37]:
#separando em dados de treinamento e validação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
models = [
    LogisticRegression(random_state=42, solver='lbfgs', multi_class='multinomial'),
    SVC(C=1,gamma='scale'),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier()]

In [45]:
for model in models:
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))

0.8111111111111111
0.8333333333333334
0.21944444444444444
0.4111111111111111
